#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 2:   word2vec + SVM + Evaluation

### 100 points [6% of your final grade]

### Due: Tuesday, February 26, 2019 by 11:59pm

*Goals of this homework:* Understand word2vec-like term embeddings,  explore real-world challenges with SVM-based classifiers, understand and implement several evaluation metrics.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw2.ipynb`. For example, my homework submission would be something like `555001234_hw2.ipynb`. Submit this notebook via eCampus (look for the homework 2 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1: Term embeddings + SVM (80 points)

### Dataset


For this homework, we will still play with Yelp reviews from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge). As in Homework 1, you'll see that each line corresponds to a review on a particular business. Each review has a unique "ID" and the text content is in the "review" field. Additionally, this time, we also offer you the "label". If `label=1`, it means that this review is `Food-relevant`. If `label=0`, it means that this review is `Food-irrelevant`. Similarly, we have already done some basic preprocessing on the reviews, so you can just tokenize each review using whitespace.

There are about 40,000 reviews in total, in which about 20,000 reviews are "Food-irrelevant". We split the review data into two sets. *review_train.json* is the training set. *review_test.json* is the testing set. 

In [1]:
# Please load the dataset
# Your code below

import json

def read_data(data_file):
    data = []
    with open(data_file) as f:
        for line in f:
            data.append(json.loads(line))
    return data

train_data = read_data('review_train.json')
test_data = read_data('review_test.json')

###  Pre-trained term embeddings

To save your time, you can make use of  pre-trained term embeddings. In this homework, we are using one of the great pre-trained models from [GloVe](https://nlp.stanford.edu/projects/glove/) based on 2 billion tweets. GloVe is quite similar to word2vec. Unzip the *glove.6B.50d.txt.zip* file and run the code below. You will be able to load the term embeddings model, with which each word can be represented with a 50-dimension vector.

In [2]:
# reload the pre-trained term embeddings
import numpy as np

with open("glove.6B.50d.txt", "rb") as lines:
    model = {line.split()[0].decode('utf-8'): np.array(list(map(float, line.split()[1:])))
           for line in lines}

Now, you have a vector representation for each word. First, we use the simple (arithmetic) **mean** of these vectors of words in a review to represent the review. *Note: Just ignore those words which are not in the corpus of this pre-trained model.*

In [3]:
# Please figure out the vector representation for each review in the training data and testing data.
# Your code below

def get_vectors(data):
    review_vector = []
    review_labels = []
    for entries in data:
        review_vector.append(np.mean([model[w] for w in entries["review"].split() if w in model] or [np.zeros(dim)], axis=0))
        review_labels.append(entries["label"])
    return (review_vector, review_labels)

train_review_vector, train_review_labels = get_vectors(train_data)
test_review_vector, test_review_labels = get_vectors(test_data)

### SVM

With the vector representations you get for each review, please train an SVM model to predict whether a given review is food-relevant or not. **You do not need to implement any classifier from scratch. You may use scikit-learn's built-in capabilities.** You can only train your model with reviews in *review_train.json*.

In [4]:
# SVM model training
# Your code here

from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(train_review_vector, train_review_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Your goal is to predict whether a given review is food-relevant or not. Please report the overall accuracy, precision and recall of your model on the **testing data**. You should **implement the functions for accuracy, precision, and recall**.

In [5]:
# Your code here
test_predict_labels = clf.predict(test_review_vector)

In [6]:
def measure_performance(actual_labels, predicted_labels):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(predicted_labels)): 
        if actual_labels[i]==predicted_labels[i]==1:
           TP += 1
        if predicted_labels[i]==1 and actual_labels[i]!=predicted_labels[i]:
           FP += 1
        if actual_labels[i]==predicted_labels[i]==0:
           TN += 1
        if predicted_labels[i]==0 and actual_labels[i]!=predicted_labels[i]:
           FN += 1
        
    print("Precision: ", TP /(TP + FP))
    print("Recall: ", TP / (TP + FN))
    print("Accuracy: ", (TP + TN)/(TP + FP + TN + FN))

In [7]:
measure_performance(test_review_labels, test_predict_labels)

Precision:  0.8721167425074533
Recall:  0.9349032800672834
Accuracy:  0.8991610738255034


### Document-based embeddings

Instead of taking the mean of term embeddings, you can directly train a **doc2vec** model for paragraph or document embeddings. You can refer to the paper [Distributed Representations of Sentences and Documents](https://arxiv.org/pdf/1405.4053v2.pdf) for more details. And in this homework, you can make use of the implementation in [gensim](https://radimrehurek.com/gensim/models/doc2vec.html).

Now, you need to:
* Train a doc2vec model based on all reviews you have (training + testing sets).
* Use the embeddings from your doc2vec model to represent each review and train a new SVM model.
* Report the overall accuracy, precision and recall of your model on the testing data.

In [8]:
# Train a doc2vec
# Your code here

import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

train_corpus = []
test_corpus = []

def read_corpus(data):
    for i,line in enumerate(data):
        yield TaggedDocument(gensim.utils.simple_preprocess(line["review"]), [line["id"]])
    
train_corpus = list(read_corpus(train_data))
test_corpus = list(read_corpus(test_data))
model_doc2vec = Doc2Vec(vector_size=50, min_count=5, workers=4, epochs=40) 
model_doc2vec.build_vocab(train_corpus + test_corpus)
model_doc2vec.train(train_corpus + test_corpus, total_examples=model_doc2vec.corpus_count, epochs=model_doc2vec.epochs)
train_vector = [model_doc2vec.docvecs[i] for i in range(0,len(train_corpus))]

In [9]:
clf = SVC(gamma='auto')
clf.fit(train_vector, train_review_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
'''
    To use LDA instead of SVM run the below code:
'''
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

#clf = LDA()
#clf.fit(train_vector, train_review_labels)

'\n    To use LDA instead of SVM run the below code:\n'

In [11]:
#performance on SVM

#test_vector = [model_doc2vec.infer_vector(entries["review"].split()) for entries in test_data]
test_vector = []
for i in range(len(train_corpus), len(train_corpus) + len(test_corpus)):
    test_vector.append(model_doc2vec.docvecs[i])

test_labels = clf.predict(test_vector)
measure_performance(test_review_labels, test_labels)

Precision:  0.9372804816859006
Recall:  0.9426408746846089
Accuracy:  0.9399328859060403


What do you observe? How different are your results for the term-based average approach vs. the doc2vec approach? Why do you think this is?

I can see that the doc2vec approach performs better than the term based average approach. The doc2vec approach provides a increase in accuracy by 5% which is significant.

doc2vec is an extension of word2vec where embeddings are constructed from entire documents (rather than the individual words). Doc2vec is a better version of word2vec because it takes word order into account, generalizes to longer documents, and can learn from unlabelled data.

### Can you do better?

Finally, see if you can do better than either the word- or doc- based embeddings approach for classification. You may explore new features, new classifiers, etc. Whatever you like. Just provide your code and a justification.

### In the word based embedding approach where we took the arithmetic mean of the word vectors, I tried to exclude stop words in english for training and testing. I can see that there is a increase in performance, because we are eliminating the words which are redundant and do not add much information. 

In [12]:
# your code here

# Please figure out the vector representation for each review in the training data and testing data.
# Your code below

from collections import defaultdict
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

train_review_vector = []
test_review_vector = []
train_review_labels = []
test_review_labels = []

def create_freq_pairs(data):
    freq_list = {}
    for entries in data:
        freq_dict = defaultdict(lambda:0)
        for word in entries["review"].split():
            freq_dict[word] += 1
        freq_list[entries["id"]] = freq_dict
    return freq_list

def get_modified_vectors(data):
    review_vector = []
    review_labels = []
    for entries in data:
        review_vector.append(np.mean([model[w] for w in entries["review"].split() if w in model and w not in stop_words] or [np.zeros(dim)], axis=0))
        review_labels.append(entries["label"])
    return (review_vector, review_labels)

freq_list = create_freq_pairs(train_data)
stop_words = set(stopwords.words('english')) 
train_review_vector, train_review_labels = get_modified_vectors(train_data)
test_review_vector, test_review_labels = get_modified_vectors(test_data)

In [13]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(train_review_vector, train_review_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
'''
    Tried using LDA instead of SVM, but SVM gives better performance.
'''

#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

#clf = LDA()
#clf.fit(train_review_vector, train_review_labels)

'\n    Tried using LDA instead of SVM, but SVM gives better performance.\n'

In [15]:
test_predict_labels = clf.predict(test_review_vector)

In [16]:
measure_performance(test_review_labels, test_predict_labels)

Precision:  0.8902380571976354
Recall:  0.9372582001682086
Accuracy:  0.9110738255033557


# Part 2: NDCG (20 points)

You calculated the recall and precision in Part 1 and now you get a chance to implement NDCG. 

Assume that Amy searches for "food-relevant" reviews in the **testing set** on two search engines `A` and `B`. Since the ground-truth labels for the reviews are unknown to A and B, they need to make a prediction for each review and then return a ranked list of results based on their probabilities. The results from A are in *search_result_A.json*, and the results from B are in *search_result_B.json*. Each line contains the id of a review and its corresponding ranking.

You can check their labels in *review_test.json* while calculating the NDCG scores. If a review is "food-relevant", the relevance score is 1. Otherwise, the relevance score is 0.

In [17]:
# NDCG for search_result_A.json
# Your code here

import math
import operator

test_dict = {}
for k in test_data:
        test_dict[k["id"]] = k["label"]
        
def parse_search_result(search_data_file):
    search_data = {}
    with open(search_data_file) as f:
        for line in f:
            search_data[line[8:30]] = int(line[40:-2])
    return search_data
        
def calculate_dcg(search_data):      
    dcg = [test_dict[x]/math.log2(search_data[x]+1) for x in search_data]
    return sum(dcg)

def calculate_idcg(p):
    idcg = [k/math.log2(i + 1) for i,k in enumerate(p,1)]
    return sum(idcg)

In [18]:
search_a_data = parse_search_result("search_result_A.json")
dcg = calculate_dcg(search_a_data)
p = sorted(test_dict.values(), reverse=True)[:len(search_a_data)]
idcg = calculate_idcg(p)
print(dcg/idcg)

0.9182854705835336


In [19]:
# NDCG for search_result_B.json
# Your code here

search_b_data = parse_search_result("search_result_B.json")
dcg = calculate_dcg(search_b_data)
p = sorted(test_dict.values(), reverse=True)[:len(search_b_data)]
idcg = calculate_idcg(p)
print(dcg/idcg)

0.9858173908934676


## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*